In [13]:
import requests
import json
import os
import time
import pandas as pd
from tqdm import tqdm
from newspaper import Article

API_KEY = "f0554c4f7fed288cdee467b475a09285ae83372b"
SEARCH_URL = "https://google.serper.dev/search"
RESULTS_PER_POLITICIAN = 10
CSV_FILE = "articles.csv"

In [14]:
# Wczytanie listy polityków
with open("politicians.txt", "r", encoding="utf-8") as f:
    politicians = [line.strip() for line in f.readlines()]
print(politicians)

['Andrzej Duda', 'Joe Biden', 'Emmanuel Macron', 'Angela Merkel', 'Boris Johnson']


In [ ]:
country_language_map = {
    "Poland": ("pl", "pl"),          
    "United States": ("us", "en"), 
    "France": ("fr", "fr"),         
    "Germany": ("de", "de"),       
    "Spain": ("es", "es"),         
    "India": ("in", "hi"),         
    "Italy": ("it", "it"),         
    "Japan": ("jp", "ja"),         
    "Brazil": ("br", "pt"),        
    "Russia": ("ru", "ru"),        
}

In [16]:
# Funkcja do pobierania artykułów
def search_articles(query, country="US", language="en"):
    headers = {
        "X-API-KEY": API_KEY,
        "Content-Type": "application/json"
    }
    
    payload = {
        "q": query,
        "num": RESULTS_PER_POLITICIAN,
        "gl": country,  # Ustawienie kraju
        "hl": language  # Ustawienie języka
    }
    
    try:
        response = requests.post(SEARCH_URL, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        data = response.json()
        return [item["link"] for item in data.get("organic", [])]
    except requests.exceptions.RequestException:
        return []



In [17]:
# Funkcja do pobierania treści artykułu
def fetch_article_content(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except Exception:
        return ""

In [18]:
# Wczytanie istniejących artykułów, aby uniknąć duplikatów
existing_links = set()
if os.path.exists(CSV_FILE):
    existing_df = pd.read_csv(CSV_FILE, usecols=["Article_Link"])
    existing_links = set(existing_df["Article_Link"].tolist())

In [19]:
politician_country = {}
with open("politicians_country.txt", "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split(",")
        if len(parts) == 2:  # Sprawdzenie poprawnego formatu
            name, country = parts
            politician_country[name] = country
print(politician_country)  # Sprawdzamy poprawność wczytanych danych

{'Andrzej Duda': ' Poland', 'Joe Biden': ' United States', 'Emmanuel Macron': ' France', 'Angela Merkel': ' Germany', 'Boris Johnson': ' United Kingdom'}


In [ ]:
# Zbieranie danych
data = []
count_good = 0  
count_bad = 0 

tqdm_desc = "Pobieranie artykułów"
for politician in tqdm(politicians, desc=tqdm_desc):
    country = politician_country.get(politician, "").strip() 
    language = country_language_map.get(country)

    if not country:
        print(f"UWAGA: Brak przypisanego kraju dla {politician}")
        continue  

    if not language:
        print(f"UWAGA: Brak przypisanego języka dla kraju '{country}'. Sprawdź country_language_map.")
        print(f"Możliwe kraje: {list(country_language_map.keys())}")  # Debugging
        continue  
    
    language = language[1]  # Pobranie kodu języka
    print(f"Przypisano dla {politician}: {country}, Język: {language}")

    articles = search_articles(politician, country, language)
    
    for link in articles:
        if link in existing_links:
            continue
        
        content = fetch_article_content(link)
        if content:
            data.append({"Politician": politician, "Country": country, "Language": language, "Article_Link": link, "Content": content})
            count_good += 1
            existing_links.add(link)  
        else:
            count_bad += 1

    time.sleep(2)  # Oczekiwanie, aby nie przekroczyć limitu API

# Jeśli mamy nowe dane to zapis do pliku CSV, tworzenie df
if data:
    df = pd.DataFrame(data)
    df.to_csv(CSV_FILE, mode='a', header=not os.path.exists(CSV_FILE), index=False, encoding="utf-8")
    df["Article_Link"] = df["Article_Link"].apply(lambda x: x[:20] + "..." if len(x) > 50 else x)
    print(f"Pobieranie zakończone. Wyniki zapisano w '{CSV_FILE}'")
else:
    print("Brak nowych artykułów do zapisania.")

print(f"Poprawnie pobrano: {count_good} artykułów")
print(f"Nie udało się pobrać: {count_bad} artykułów")



Pobieranie artykułów:   0%|          | 0/5 [00:00<?, ?it/s]

Przypisano dla Andrzej Duda: Poland, Język: pl


Pobieranie artykułów:  20%|██        | 1/5 [00:04<00:18,  4.65s/it]

Przypisano dla Joe Biden: United States, Język: en


Pobieranie artykułów:  40%|████      | 2/5 [00:12<00:20,  6.67s/it]

...

Przypisano dla Angela Merkel: Germany, Język: de


Pobieranie artykułów:  80%|████████  | 4/5 [00:28<00:07,  7.43s/it]

Przypisano dla Boris Johnson: United Kingdom, Język: en


Pobieranie artykułów: 100%|██████████| 5/5 [00:35<00:00,  7.12s/it]

Pobieranie zakończone. Wyniki zapisano w 'articles.csv'
Poprawnie pobrano: 27 artykułów
Nie udało się pobrać: 11 artykułów
